In [ ]:
import pandas as pd

In [ ]:
news = pd.read_excel("FA-KES-Dataset.xls")
news.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,2017-05-04 00:00:00,idlib,0.0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,2017-07-04 00:00:00,homs,0.0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0.0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0.0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,2016-10-07 00:00:00,aleppo,0.0


In [ ]:
news["labels"].unique()

array([ 0.,  1., nan])

In [ ]:
news.dropna(inplace=True)
news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 803
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unit_id          755 non-null    int64  
 1   article_title    755 non-null    object 
 2   article_content  755 non-null    object 
 3   source           755 non-null    object 
 4   date             755 non-null    object 
 5   location         755 non-null    object 
 6   labels           755 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 47.2+ KB


In [ ]:
y = news["labels"].values
X = (news["article_title"] + " " + news["article_content"]).values

In [ ]:
X[0]

'Syria attack symptoms consistent with nerve agent use WHO Wed 05 Apr 2017 Syria attack symptoms consistent with nerve agent use WHO. Victims of a suspected chemical attack in Syria appeared to show symptoms consistent with reaction to a nerve agent the World Health Organization said on Wednesday. "Some cases appear to show additional signs consistent with exposure to organophosphorus chemicals a category of chemicals that includes nerve agents" WHO said in a statement putting the death toll at at least 70. The United States has said the deaths were caused by sarin nerve gas dropped by Syrian aircraft. Russia has said it believes poison gas had leaked from a rebel chemical weapons depot struck by Syrian bombs. Sarin is an organophosporus compound and a nerve agent. Chlorine and mustard gas which are also believed to have been used in the past in Syria are not. A Russian Defence Ministry spokesman did not say what agent was used in the attack but said the rebels had used the same chemic

In [ ]:
X.shape, y.shape

((755,), (755,))

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
train_texts.shape, test_texts.shape

((679,), (76,))

In [ ]:
type(train_texts[0])

NameError: name 'train_texts' is not defined

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments


tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encondings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.lables[idx])
    return item

  def __len__(self):
    return len(self.labels)


train_dateset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)